In [128]:
import sys
sys.path.append("..")
import pandas as pd
from app import kernel
from app import smiles
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn.manifold import SpectralEmbedding
from sklearn.gaussian_process import GaussianProcessRegressor
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('../data/result-ML-All-critical.txt', sep=' ')

graph_list = [smiles.smiles2graph(i) for i in df['SMILES']]
df['graph'] = graph_list

In [136]:
graph_list_n = np.array(graph_list[:1000])

# direct computation of kernel

In [118]:
%timeit Knn_direct_3000 = graph_kernel(graph_list)

29.1 s ± 434 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [124]:
%timeit Knn_direct_2000 = graph_kernel(graph_list[:2000])

7.7 s ± 9.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [120]:
%timeit Knn_direct_1000 = graph_kernel(graph_list_n)

1.74 s ± 9.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [123]:
%timeit Knn_direct_500 = graph_kernel(graph_list_n[np.random.randint(0,1000,500)])

442 ms ± 5.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# random projection of m=100

In [121]:
%timeit Kmm = graph_kernel(graph_list_m, graph_list_m)

229 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [217]:
idx_list_m = []
for i in range(10):
    idx_list_m.append(cluster_old(graph_list_n[100*i:100*i+100], 20))

In [160]:
graph_list_m = cluster_old(graph_list_n[np.array(idx_list_m).reshape(-1)])

In [180]:
Kmm[Kmm > 0.999] = 0

In [181]:
Kmm

array([[0.        , 0.4188233 , 0.7606968 , ..., 0.90628403, 0.48916414,
        0.4258283 ],
       [0.4188233 , 0.        , 0.6758113 , ..., 0.26975504, 0.9038005 ,
        0.96963274],
       [0.7606969 , 0.6758113 , 0.        , ..., 0.5749801 , 0.6511051 ,
        0.62904036],
       ...,
       [0.90628386, 0.269755  , 0.5749801 , ..., 0.        , 0.36811525,
        0.29391584],
       [0.48916414, 0.9038004 , 0.6511051 , ..., 0.36811528, 0.        ,
        0.93660164],
       [0.4258283 , 0.96963274, 0.62904036, ..., 0.2939158 , 0.93660164,
        0.        ]], dtype=float32)

In [182]:
(Kmm > 0.9).any()

True

In [161]:
Knm = graph_kernel(graph_list_n, graph_list_m)
Kmm = graph_kernel(graph_list_m, graph_list_m)
Kmm_inv = np.linalg.inv(Kmm)
Knn_rp = Knm.dot(Kmm_inv).dot(Knm.T)

In [47]:
%timeit np.linalg.inv(Kmm)

113 µs ± 1.83 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [48]:
%timeit np.linalg.pinv(Kmm)

1.03 ms ± 17.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [82]:
Knn.shape

(1000, 1000)

In [83]:
Knn_direct = graph_kernel(graph_list_n)

In [162]:
Knn_rp

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [129]:
def cluster_old(X, add_size=10):
    ''' find representative samples from a pool using clustering method
    :X: a list of graphs
    :add_sample_size: add sample size
    :return: list of idx
    '''
    # train SpectralClustering on X
    if len(X) < add_size:
        return [ i for i in range( len(X))]
    gram_matrix = graph_kernel(X)
    result = SpectralClustering(n_clusters=add_size, affinity='precomputed').fit_predict(gram_matrix) # cluster result
    # distance matrix
    #distance_mat = np.empty_like(gram_matrix)
    #for i in range(len(X)):
    #    for j in range(len(X)):
    #        distance_mat[i][j] = np.sqrt(abs(gram_matrix[i][i] + gram_matrix[j][j] - 2 * gram_matrix[i][j]))
    # choose the one with least in cluster distance sum in each cluster
    total_distance = {i:{} for i in range(add_size)} # (key: cluster_idx, val: dict of (key:sum of distance, val:idx))
    for i in range(len(X)): # get all in-class distance sum of each item
        cluster_class = result[i]
        #total_distance[cluster_class][np.sum((np.array(result) == cluster_class) * distance_mat[i])] = i
        total_distance[cluster_class][np.sum((np.array(result) == cluster_class) * 1/gram_matrix[i])] = i
    add_idx = [total_distance[i][min(total_distance[i].keys())] for i in range(add_size)] # find min-in-cluster-distance associated idx
    return add_idx

In [251]:
graph_subset = graph_list_n[np.random.randint(0,1000,400)]
Kmm = graph_kernel(graph_subset[np.array(cluster_old(graph_subset, 40))])

In [252]:
((np.linalg.inv(Kmm).dot(Kmm) - np.eye(40))**2).mean()

3.573724051744206e-11

In [241]:
Kmm_2 = graph_kernel(graph_list_n[np.random.randint(0,1000,40)])

In [242]:
((np.linalg.inv(Kmm_2).dot(Kmm_2) - np.eye(40))**2).sum()


5.697758773109678e-08

In [8]:
df.head()

,SMILES,tc,dc,graph
0,CCCC,414.5,0.2269,"<GraphE(nodes={'!i': [0,1,2,3], 'aromatic': [F..."
1,CC(C)C,407.2,0.2291,"<GraphE(nodes={'!i': [0,1,2,3], 'aromatic': [F..."
2,CCCCC,459.5,0.2320,"<GraphE(nodes={'!i': [0,1,2,3,4], 'aromatic': ..."
3,CCC(C)C,455.0,0.2328,"<GraphE(nodes={'!i': [0,1,2,3,4], 'aromatic': ..."
4,CC(C)(C)C,435.9,0.2366,"<GraphE(nodes={'!i': [0,1,2,3,4], 'aromatic': ..."


# simple alpha test

In [142]:
df = pd.read_csv('../data/result-ML-All-critical.txt', sep=' ')

graph_list = [smiles.smiles2graph(i) for i in df['SMILES']]
df['graph'] = graph_list

In [153]:
from graphdot.kernel.marginalized import MarginalizedGraphKernel
from graphdot.kernel.basekernel import TensorProduct
from graphdot.kernel.basekernel import SquareExponential
from graphdot.kernel.basekernel import KroneckerDelta

class NormalizedKernelGrad(MarginalizedGraphKernel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __normalize(self, X, Y, R):
        if Y is None:
            # square matrix
            if type(R) is tuple:
                d = np.diag(R[0]) ** -0.5
                K = np.diag(d).dot(R[0]).dot(np.diag(d))
                K_gradient = np.einsum("ijk,i,j->ijk", R[1], d, d)
                return K, R[1]# K_gradient
            else:
                d = np.diag(R) ** -0.5
                K = np.diag(d).dot(R).dot(np.diag(d))
                return K
        else:
            # rectangular matrix, must have X and Y
            if type(R) is tuple:
                diag_X = super().diag(X) ** -0.5
                diag_Y = super().diag(Y) ** -0.5
                K = np.diag(diag_X).dot(R[0]).dot(np.diag(diag_Y))
                K_gradient = np.einsum("ijk,i,j->ijk", R[1], diag_X, diag_Y)
                return K, R[1] #K_gradient
            else:
                diag_X = super().diag(X) ** -0.5
                diag_Y = super().diag(Y) ** -0.5
                K = np.diag(diag_X).dot(R).dot(np.diag(diag_Y))
                return K

    def __call__(self, X, Y=None, *args, **kwargs):
        R = super().__call__(X, Y, *args, **kwargs)
        return self.__normalize(X, Y, R)

class NormalizedKernel(MarginalizedGraphKernel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __normalize(self, X, Y, R):
        if Y is None:
            # square matrix
            if type(R) is tuple:
                d = np.diag(R[0]) ** -0.5
                K = np.diag(d).dot(R[0]).dot(np.diag(d))
                K_gradient = np.einsum("ijk,i,j->ijk", R[1], d, d)
                return K, K_gradient
            else:
                d = np.diag(R) ** -0.5
                K = np.diag(d).dot(R).dot(np.diag(d))
                return K
        else:
            # rectangular matrix, must have X and Y
            if type(R) is tuple:
                diag_X = super().diag(X) ** -0.5
                diag_Y = super().diag(Y) ** -0.5
                K = np.diag(diag_X).dot(R[0]).dot(np.diag(diag_Y))
                K_gradient = np.einsum("ijk,i,j->ijk", R[1], diag_X, diag_Y)
                return K # , K_gradient
            else:
                diag_X = super().diag(X) ** -0.5
                diag_Y = super().diag(Y) ** -0.5
                K = np.diag(diag_X).dot(R).dot(np.diag(diag_Y))
                return K

    def __call__(self, X, Y=None, *args, **kwargs):
        R = super().__call__(X, Y, *args, **kwargs)
        return self.__normalize(X, Y, R)
    
knode = TensorProduct(aromatic=KroneckerDelta(0.8),
                      #charge=SquareExponential(1.0),
                      element=KroneckerDelta(0.5),
                      hcount=SquareExponential(1.0)
                      )
kedge = TensorProduct(order=SquareExponential(1.0),
                      stereo=KroneckerDelta(0.8),
                      conjugated=KroneckerDelta(0.8),
                      inring=KroneckerDelta(0.8)
                      )
graph_kernel = NormalizedKernel(knode, kedge, q=0.05)
graph_kernel_grad = NormalizedKernelGrad(knode, kedge, q=0.05)

In [154]:
np.random.seed(233)
model = GaussianProcessRegressor(kernel=graph_kernel_grad, random_state=0,
                                normalize_y=True, alpha=0.5).fit(df['graph'][np.random.choice(1000,500)], df['dc'][np.random.choice(1000,500)])

In [155]:
y_pred = model.predict(df['graph'][600:700])
# R2
r2 = r2_score(y_pred, df['dc'][600:700])
# MSE
mse = mean_squared_error(y_pred, df['dc'][600:700])

In [156]:
r2, mse

(-8.684696398703204, 0.00027194021404459143)

In [149]:
len(df)

3617

In [140]:
len(df)

1337